In [1]:
# Import delle librerie necessarie
import torch
import numpy as np
import random
import pandas as pd
import wandb
import os

print("Librerie caricate.")

Librerie caricate.


### Impostazioni per la riproducibilità 

In [2]:
def set_seed(seed_value=42):
    """Imposta i seed per la riproducibilità."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        # Imposta anche i seed per la GPU, se disponibile
        torch.cuda.manual_seed_all(seed_value)

# Esegui l'impostazione del seed
set_seed(42) 
print("Random seeds impostati su 42.")

Random seeds impostati su 42.


### Configurazione Iniziale degli Hyperparameters (W&B)

Questa sezione definisce i principali parametri di addestramento (*hyperparameters*) per il *fine-tuning* del modello **ModernBERT (RoBERTa-base)**, registrandoli in **Weights & Biases** per tracciabilità e riproducibilità.

| Parametro | Valore | Motivazione della Scelta |
| :--- | :--- | :--- |
| **`learning_rate`** | `5e-5` (0.00005) | È il tasso di apprendimento *default* consigliato da Google/Hugging Face per il *fine-tuning* dei modelli BERT/RoBERTa. È un valore conservativo che assicura che il modello si adatti al nuovo *task* (ACOS) senza "dimenticare" le conoscenze acquisite nel *pre-training*. |
| **`epochs`** | `5` | Un valore tipico e contenuto per il *fine-tuning* di modelli Transformer. Di solito, 3 o 4 *epochs* sono sufficienti per convergere, ma 5 offrono un buon equilibrio tra addestramento e prevenzione dell'overfitting sui dataset di dimensioni limitate come ACOS. |
| **`batch_size`** | `16` | Questa dimensione del *batch* è comune quando si lavora con modelli grandi come RoBERTa-base, bilanciando la stabilità dell'addestramento con le limitazioni della **memoria GPU**. |
| **`model_name`** | `roberta-base` | Scegliamo **RoBERTa-base** (un'architettura *Encoder-only*) come modello **"ModernBERT"** in quanto offre prestazioni superiori e un *pre-training* più robusto rispetto al **BERTbase** originale, come richiesto dal professore per superare la *baseline* del paper ACOS. |
| **`dataset`** | `Laptop-ACOS` | Identifica il sotto-dataset specifico utilizzato per questo esperimento. |
| **`seed`** | `42` | Il **seed di riproducibilità**, fissato a 42 (la convenzione standard ML), garantisce che ogni volta che lo *script* viene eseguito, l'inizializzazione dei pesi e lo *shuffling* dei dati siano identici, garantendo la tracciabilità scientifica dei risultati. |

---

In [5]:
WANDB_ENTITY = "cristinatextmining"

# 1. Definizione degli Hyperparameters
config = {
    "learning_rate": 5e-5,
    "epochs": 5,
    "batch_size": 16,
    "model_name": "roberta-base", # Il tuo ModernBERT
    "dataset": "Laptop-ACOS",
    "seed": 42
}

# 2. Inizializzazione del Run
wandb.init(
    project="BigData-TextMining-ACOS",
    entity=WANDB_ENTITY,
    config=config,
    name=f"run_{config['model_name']}_{config['dataset']}"
)

print(f"W&B inizializzato per il progetto: {wandb.run.project}")

W&B inizializzato per il progetto: BigData-TextMining-ACOS


### Parsing Avanzato per l'Estrazione di Span (ACOS)
Il dataset ACOS utilizza un formato complesso per l'annotazione, tipico dei compiti di Sequence Labeling o Estrazione di Span, dove i dati target (le quadruple) sono mescolati con la frase.

L'obiettivo di questo parsing sarà di trasformare ogni riga grezza in una struttura pulita contenente:
* review_text: Il testo puro e completo della recensione (l'input del modello).
* quadruples_list: Una lista di stringhe di quadruple codificate (il target che dovrà essere processato successivamente in span).
